# Chapter 11 인터페이스: 프로토콜에서 ABC까지
**동적 자료형**

> 자료형 검사의 대부분이 컴파일 타임이 아닌 실행 시간에 수행될 경우 동적 정형이라고 한다. 동적 정형에서, 값은 자료형을 가지고 있지만 변수는 그렇지 않다. 즉, 변수는 모든 자료형의 값을 가질 수 있다. [https://ko.wikipedia.org/wiki/자료형_체계#동적_정형](https://ko.wikipedia.org/wiki/%EC%9E%90%EB%A3%8C%ED%98%95_%EC%B2%B4%EA%B3%84#%EB%8F%99%EC%A0%81_%EC%A0%95%ED%98%95)

<br>

**인터페이스**
> 시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부 

<br>

**덕타이핑**

> 컴퓨터 프로그래밍 분야에서 덕 타이핑(duck typing)은 동적 타이핑의 한 종류로, 객체의 변수 및 메소드의 집합이 객체의 타입을 결정하는 것을 말한다. 클래스 상속이나 인터페이스 구현으로 타입을 구분하는 대신, 덕 타이핑은 객체가 어떤 타입에 걸맞은 변수와 메소드를 지니면 객체를 해당 타입에 속하는 것으로 간주한다. [https://ko.wikipedia.org/wiki/덕_타이핑](https://ko.wikipedia.org/wiki/%EB%8D%95_%ED%83%80%EC%9D%B4%ED%95%91)

- 객체의 실제 자료형은 무시하고, 객체가 용도에 맞는 메서드 이름, 시그니처, 의미를 구현하도록 보장하는 데 주안점을 둔다. 


<br>

**동적 프로토콜**
- 어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스
  - 객체의 정체 (자료형)가 중요한 것이 아니다. 객체가 해당 역할을 수행할 수 있는 기능 (메서드)이 있는지가 중요한 것이다.
- 객체가 어떤 프로토콜을 구현하는 한 자료형에 상관 없이 객체를 작동시킨다. 
  - 예) 함수가 인자로 들어온 객체의 자료형에 신경 쓰지 않는다. 객체가 일부 프로토콜을 구현하고 있으면 된다.


<br>

**추상 베이스 클래스 (abc)**

> 베이스 클래스를 상속 받는 파생 클래스가 반드시 베이스 클래스의 메서드를 명시적으로 선언해서 구현하도록 강제하는 추상화 클래스 [https://bluese05.tistory.com/61](https://bluese05.tistory.com/61)

- 각 메서드에 일일이 `NotImplementedError`를 붙이는 것과의 차이점
  - abc 클래스를 사용할 경우 베이스 클래스는 인스턴스화될 수 없다.
  - 메서드가 실행될 때 에러가 발생하는 `NotImplementedError`와 달리, 객체 생성에서 에러가 발생한다.


<br>

## 11.1 파이썬 문화에서의 인터페이스와 프로토콜

<br>

## 11.2 파이썬은 시퀀스를 찾아낸다.
- `collections.abs`에 정의된 `Sequence ABC`에 따르면,  `__len__()`, `__getitem()__` 메서드가 구현되어야 `Sequence`형이 된다.
  - `__contains__()`, `__iter__()`는 Sequence 추상 클래스의 구상 메서드로 구현되어 있기 때문에 위 2개만 정의하면 된다.
- 하지만, 시퀀스 프로토콜의 중요성 때문에 `__iter__()`와 `__contains__()` 메서드가 구현되어 있지 않더라도, 파이썬은 `__getitem__()` 메서드를 호출해서 객체를 반복하고 `in` 연산자를 사용할 수 있게 해준다.

In [1]:
from collections.abc import Sequence

class MySeq(Sequence):
    def __getitem__(self, index):
        print(index)
        
a = MySeq()

TypeError: Can't instantiate abstract class MySeq with abstract methods __len__

In [2]:
# 예제 11-3 __getitem__()으로 부분 구현한 시퀀스 프로토콜. 항목에 접근할 수 있고, 반복할 수 있으며, in 연산자에서 사용할 수 있다.
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]
    
f = Foo()
print(f[1])

for i in f:
    print(i)
    
print(20 in f)
print(15 in f)

10
0
10
20
True
False


In [3]:
class Boo:
    pass

Boo.__dict__
b = Boo()

3 in b

TypeError: argument of type 'Boo' is not iterable

In [4]:
Foo.__dict__.

SyntaxError: invalid syntax (<ipython-input-4-712cbcf29003>, line 1)

In [5]:
# 예제 11-4: 일련의 카드로 구성된 카드 한 벌
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])  # 개별 카드를 나타내는 클래스


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')  # 인스턴스로 생성되지 않아도 접근 가능 (ex) FrenchDeck.ranks
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]


## 11.3 런타임에 프로토콜을 구현하는 멍키 패칭


In [6]:
from random import shuffle

l = list(range(10))
shuffle(l)
print(l)

[4, 6, 0, 7, 9, 1, 8, 2, 5, 3]


In [7]:
# 예제 11-5 random.shuffle()이 FrenchDeck을 처리하지 못한다.
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [8]:
deck[0] = 0

TypeError: 'FrenchDeck' object does not support item assignment

- 객체 할당을 지원하기 위해서는 `__setitem__()` 메세드를 구현해야 한다.
- 파이썬은 동적 언어이기 때문에, 대화형 콘솔에서 실행하는 동안에도 이 문제를 해결할 수 있다 (멍키 패칭).

In [9]:
# 예제 11-6 FrenchDeck을 가변형으로 만들어 random.shuffle()을 사용하기 위한 멍키 패칭
def set_card(deck, position, card):
    deck._cards[position] = card
    
FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='7', suit='clubs'),
 Card(rank='9', suit='spades'),
 Card(rank='J', suit='hearts'),
 Card(rank='9', suit='hearts'),
 Card(rank='K', suit='diamonds')]

In [10]:
from collections import abc

def type_checker(obj):
    print("Is abc.Container? ", isinstance(obj, abc.Container))
    print("Is abc.Iterable? ", isinstance(obj, abc.Iterable))
    print("Is abc.Sized? ", isinstance(obj, abc.Sized))
    print("Is abc.Sequence? ", isinstance(obj, abc.Sequence))

In [11]:
class MySequence:
    def __init__(self, data):
        self._data = list(data)
        
    def __len__(self):
        return len(self._data)
    
my_seq = MySequence([1, 2, 3, 4])

type_checker(my_seq)

Is abc.Container?  False
Is abc.Iterable?  False
Is abc.Sized?  True
Is abc.Sequence?  False


In [12]:
class MySequence:
    def __init__(self, data):
        self._data = list(data)
        
    def __iter__(self):
        return (x for x in self._data)
    
my_seq = MySequence([1, 2, 3, 4])

type_checker(my_seq)

Is abc.Container?  False
Is abc.Iterable?  True
Is abc.Sized?  False
Is abc.Sequence?  False


In [13]:
class MySequence:
    def __init__(self, data):
        self._data = list(data)
        
    def __getitem__(self, index):
        return self._data[index]
    
my_seq = MySequence([1, 2, 3, 4])

type_checker(my_seq)

Is abc.Container?  False
Is abc.Iterable?  False
Is abc.Sized?  False
Is abc.Sequence?  False


In [14]:
class MySequence:
    def __init__(self, data):
        self._data = list(data)
        
    def __getitem__(self, index):
        return self._data[index]
    
    def __len__(self):
        return len(self._data)
    
my_seq = MySequence([1, 2, 3, 4])

type_checker(my_seq)

Is abc.Container?  False
Is abc.Iterable?  False
Is abc.Sized?  True
Is abc.Sequence?  False


In [15]:
class MySequence:
    def __init__(self, data):
        self._data = list(data)
        
    def __contains__(self, value):
        return value in self._data
    
    def __iter__(self):
        return (x for x in self._data)
        
    def __getitem__(self, index):
        return self._data[index]
    
    def __len__(self):
        return len(self._data)
    
my_seq = MySequence([1, 2, 3, 4])

type_checker(my_seq)

Is abc.Container?  True
Is abc.Iterable?  True
Is abc.Sized?  True
Is abc.Sequence?  False


In [16]:
class MySequence:
    def __init__(self, data):
        self._data = list(data)
        
    def __contains__(self, value):
        return value in self._data
    
    def __iter__(self):
        return (x for x in self._data)
        
    def __getitem__(self, index):
        return self._data[index]
    
    def __len__(self):
        return len(self._data)
    
    def __reversed__(self):
        return self._data[::-1]
    
my_seq = MySequence([1, 2, 3, 4])

type_checker(my_seq)

Is abc.Container?  True
Is abc.Iterable?  True
Is abc.Sized?  True
Is abc.Sequence?  False


In [17]:
class MySequence:
    def __init__(self, data):
        self._data = list(data)
        
    def __contains__(self, value):
        return value in self._data
    
    def __iter__(self):
        return (x for x in self._data)
        
    def __getitem__(self, index):
        return self._data[index]
    
    def __len__(self):
        return len(self._data)
    
    def __reversed__(self):
        return self._data[::-1]
    
my_seq = MySequence([1, 2, 3, 4])

type_checker(my_seq)

Is abc.Container?  True
Is abc.Iterable?  True
Is abc.Sized?  True
Is abc.Sequence?  False


In [18]:
class MySequence:
    def __init__(self, data):
        self._data = list(data)
        
    def __contains__(self, value):
        return value in self._data
    
    def __iter__(self):
        return (x for x in self._data)
        
    def __getitem__(self, index):
        return self._data[index]
    
    def __len__(self):
        return len(self._data)
    
    def __reversed__(self):
        return self._data[::-1]
    
    def index(self, index):
        return self._data.index(index)
    
    def count(self, value):
        return self._data.count(value)
    
my_seq = MySequence([1, 2, 3, 4])

type_checker(my_seq)

Is abc.Container?  True
Is abc.Iterable?  True
Is abc.Sized?  True
Is abc.Sequence?  False


In [19]:
class MySequence(abc.Sequence):
    def __init__(self, data):
        self._data = list(data)
        
    def __contains__(self, value):
        return value in self._data
    
    def __iter__(self):
        return (x for x in self._data)
        
    def __getitem__(self, index):
        return self._data[index]
    
    def __len__(self):
        return len(self._data)
    
    
my_seq = MySequence([1, 2, 3, 4])

type_checker(my_seq)

Is abc.Container?  True
Is abc.Iterable?  True
Is abc.Sized?  True
Is abc.Sequence?  True


## 11.4 알렉스 마르텔리의 물새


## 11.5 ABC 상속하기
- `collections.MutableSequence`의 세 가지 추상 메서드 `__setitem__()`, `__delitem__()`, `insert()`
- 컴파일 또는 모듈 임포트 단계가 아니라 인스턴스를 생성할 때 추상 메서드 구현 여부를 확인한다.

In [20]:
# 예제 11-8 collections.MutableSequence의 서브클래스 FrenchDeck2
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split(' ')
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, index):
        return self._cards[index]
    
    def __setitem__(self, index, value):
        self._cards[index] = value
        
    def __delitem__(self, index):
        del self._cards[index]
        
    def insert(self, index, value):
        self._cards.insert(index, value)

<ipython-input-20-5154b2182384>:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  class FrenchDeck2(collections.MutableSequence):


## 11.6 표준 라이브러리의 ABC
### 11.6.1 `collections.abc`의 ABC
- 파이썬 3.4 버전에는 16개의 ABC가 있다 (현재는 25개).
- Iterable, Container, Sized
  - 모든 컬렉션은 이 ABC를 상속하거나, 적어도 호환되는 프로토콜을 구현해야 한다.
  - 각각 `__iter__()`, `__contains__()`, `__len__()`을 통해 반복, `in` 연산자, `len()` 메서드를 지원
- Sequence, Mapping, Set
  - 불변 컬렉션로서 각자 가변형 서브 클래스 MutableSequence, MutableMapping, MutableSet이 있다.
- MappingView
  - `items()`, `keys()`, `values()` 메서드에서 반환된 객체는 각각 `ItemsView`, `KeysView`, `ValuesView`를 상속 받는다.
  - `ItemsView`, `KeysView`는 `Set`을 상속한다.
- Callable, Hashable
  - 상속 목적보다는 주로 어떤 객체를 호출하거나 해시할 수 있는지 안전하게 판단하게 위해 `isinstance` 함수에 사용된다.
- Iterator
  - Iterable을 상속하며, 자세한 내용은 14장에서 배운다.

<br>

---

### 11.6.2 ABC의 숫자탑
`numbers` 패키지에서는 소위 '숫자탑'이라고 하는 것을 정의함
- `Number`
- `Complex`
- `Real`
- `Rational`
- `Integral`

<br>

---

### 11.6.2 `abc.ABC`
- 사용자 정의 ABC를 만들 때 사용

## 11.7 ABC의 정의와 사용
예제 상황 설명
- **상황** 광고를 무작위 순으로 보여주되, 모든 광고를 다 모두 보여주기 전까지는 같은 광고를 반복하면 안됨 => **무반복 무작위**
- **전략** 집합이 소진될 때까지 반복하지 않고 유한 집합에서 무작위로 항목을 선택 => **스택**과 **큐**
- **Tombola** 무반복 무작위 선발을 지원하는 클래스
  - 2가지 추상 메서드
    - `load()`: 항목을 컨테이너 안에 넣는다.
    - `pick()`: 컨테이너 안에서 무작위로 항목 하나를 꺼내서 반환한다.
  - 2가지 구상 메서드
    - `loaded()`: 컨테이너 안에 항목이 하나 이상 있으면 True를 반환
    - `inspect()`: 내용물을 변경하지 않고 항목들을 정렬하여 튜플로 반환
    - ABC에도 구상 메서드가 들어갈 수 있으며, ABC (해당 ABC에 정의된 다른 구상/추상 메서드 혹은 프로퍼티)에 정의된 인터페이스만 사용해야 한다.

In [21]:
# 예제 11-9 tombola.py: 추상 메서드 두 개와 구상 메서드 두개를 가진 Tombola ABC
import abc

class Tombola(abc.ABC):  # ABC를 정의하려면 abc.ABC를 상속해야 한다.
    
    @abc.abstractmethod  # 추상 메서드임을 데커레이터로 표시한다. docstring만 넣는 경우가 종종 있다.
    def load(self, iterable):
        """iterable의 항목들을 추가한다."""
    
    @abc.abstractmethod
    def pick(self):  # docstring을 통해 LookupError를 발생시키라고 알려준다.
        """무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 때 이 메서드를 실행하면 `LookupError`가 발생한다.
        """
    
    def loaded(self):  # ABC에도 구상 메서드가 들어갈 수 있다. 구상 메서드들은 반드시 ABC에 정의된 인터페이스만 사용해야 한다.
        """최소 한 개의 항목이 있으면 True를, 아니면 False를 반환한다."""
        return bool(self.inspect())
    
    def inspect(self):
        """현재 안에 있는 항목들로 구성된 정렬된 튜플을 반환한다."""
        items = []
        while True:  # 서브 클래스가 어떻게 항목을 저장하는지 모르기 때문에 이렇게 정의한다.
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

- `LookupError`
  - 서브 클래스가 사용할 데이터 구조에 따라 `IndexError` 또는 `KeyError`가 발생할 것이다.
  - `LookupError`가 `IndexError`와 `KeyError`의 상위 예외이다.
  - > `exception LookupError` 매핑 또는 시퀀스에 사용된 키 나 인덱스가 잘못되었을 때 발생하는 예외의 베이스 클래스
  - **(참고)** [파이썬 예외 계층 구조](https://docs.python.org/ko/3/library/exceptions.html#exception-hierarchy)

<br>

~~~
# 예제 11-10 Exception 클래스 계층구조 일부
BaseException
 ├── BaseExceptionGroup
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
      ... 중략 ...
      ├── ImportError
      │    └── ModuleNotFoundError
      ├── LookupError
      │    ├── IndexError
      │    └── KeyError
      ... 후략 ...
~~~

- 서브 클래스의 추상 메서드를 정의하지 않으면 에러가 발생한다. 
- 에러 발생 시점은 인스턴스가 만들어질 때이다.

In [22]:
# 예제 11-11 들켜버린 가짜 Tombola
from tombola import Tombola

class Fake(Tombola):
    def pick(self):
        return 13
    
print(Fake)
f = Fake()

<class '__main__.Fake'>


TypeError: Can't instantiate abstract class Fake with abstract methods load

### 11.7.1 ABC 상세 구문
- 사용자 정의 ABC를 만들 때는 `abc.ABC` 또는 다른 ABC를 상속하는 방법이 가장 좋다.
  - (참고) 파이썬 3.4 이전은 metaclass 키워드를 사용하여 `abc.ABCMeta`를 상속해야 했다. `class Tombola(metaclass=abc.ABCMeta):`
  - (참고) 파이썬 2에서는 `__metaclass__` 클래스 속성을 사용해야 했다. `__metaclass__ = abc.ABCMeta`
  - (참고) 기존의 `@staticmethod`, `@classmethod`, `@property` 데코레이터를 사용하고 싶을 경우 `@abc.abstractmethod`를 제일 안쪽에 위치시키고 그 위에 중첩하여 사용하면 된다.


### 11.7.2 Tombola ABC 상속하기
- `BingoCage` 더 좋은 난수생성기를 사용

- `random.SystemRandom`는 `os.urandom()` 함수를 기반으로 random API를 구현현다. `os.urandom()`는 암호화에 적합한 무작위 bytes를 생성할 수 있다.
  - `os.urandom()`은 시드 설정을 할 수 없다. and draws its source of entropy from many unpredictable sources, making it more random.
  - [https://stackoverflow.com/a/47515179/10258799](https://stackoverflow.com/a/47515179/10258799)

In [23]:
import os
import random

print(os.urandom(1))

rng = random.SystemRandom()
print(rng.random())
print(rng.randint(0, 10))

b'\xa0'
0.7068288512139356
1


In [24]:
# 예제 11-12 Tombola의 구상 서브클래스 BingoCage
import random

from tombola import Tombola

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        self.pick()

In [25]:
ConcreteTombola = BingoCage


# 반복형에서 객체를 생성하고 로딩
balls = list(range(3))
globe = ConcreteTombola(balls)
print(globe.loaded())
print(globe.inspect())
    
# 공을 꺼내서 수집
picks = []
picks.append(globe.pick())
picks.append(globe.pick())
picks.append(globe.pick())

# 상태와 결과를 확인
print(globe.loaded())
print(sorted(picks) == balls)

# 다시 로딩
globe.load(balls)
print(globe.loaded())
picks = [globe.pick() for i in balls]
print(globe.loaded())

# 객체가 비어 있을 때 `LookupError` 또는 서브 예외가 발생하는지 확인
globe = ConcreteTombola([])
try:
    globe.pick()
except LookupError as exc:
    print('OK')
    
# 공 100개를 로딩하고 꺼내서 모두 제대로 나오는지 검증한다.
balls = list(range(100))
globe = ConcreteTombola(balls)
picks = []
while globe.inspect():
    picks.append(globe.pick())
print(len(picks) == len(balls))
print(set(picks) == set(balls))

# 순서가 변경되었는지, 단지 앞에서 뒤로 순서만 바꾼 것은 아닌지 확인한다.
print(picks != balls)
print(picks[::-1] != balls)

True
(0, 1, 2)
False
True
True
False
OK
True
True
True
True


In [26]:
# 예제 11-13 Tombola의 inspect()와 loaded() 메서드를 오버라이드하는 LotteryBlower 구상 서브클래스
import random

from tombola import Tombola

class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty BingoCage')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

In [27]:
ConcreteTombola = LotteryBlower


# 반복형에서 객체를 생성하고 로딩
balls = list(range(3))
globe = ConcreteTombola(balls)
print(globe.loaded())
print(globe.inspect())
    
# 공을 꺼내서 수집
picks = []
picks.append(globe.pick())
picks.append(globe.pick())
picks.append(globe.pick())

# 상태와 결과를 확인
print(globe.loaded())
print(sorted(picks) == balls)

# 다시 로딩
globe.load(balls)
print(globe.loaded())
picks = [globe.pick() for i in balls]
print(globe.loaded())

# 객체가 비어 있을 때 `LookupError` 또는 서브 예외가 발생하는지 확인
globe = ConcreteTombola([])
try:
    globe.pick()
except LookupError as exc:
    print('OK')
    
# 공 100개를 로딩하고 꺼내서 모두 제대로 나오는지 검증한다.
balls = list(range(100))
globe = ConcreteTombola(balls)
picks = []
while globe.inspect():
    picks.append(globe.pick())
print(len(picks) == len(balls))
print(set(picks) == set(balls))

# 순서가 변경되었는지, 단지 앞에서 뒤로 순서만 바꾼 것은 아닌지 확인한다.
print(picks != balls)
print(picks[::-1] != balls)

True
(0, 1, 2)
False
True
True
False
OK
True
True
True
True


### 11.7.3 Tombola의 가상 서브클래스
- 사용자 정의 ABC를 상속하지 않아도, register를 사용해서 가상 서브클래스로 등록할 수 있으며, 해당 클래스가 ABC에 정의된 인터페이스를 충실히 구현한다고 약속하는 것.
  - 객체가 생성될 때 파이썬은 해당 클래스가 인터페에시를 구현했는지 검사하지 않는다. 
  - register에 의해 등록된 클래스는 `issubclass()`와 `isinstance()` 함수에 의해 인식된다.
  - 메서드나 속성은 상속받은 것이 아니라 직접 구현한 것이다. -> super 안 되겠지?

In [28]:
# 예제 11-14 Tombola의 가성 서브클래스 TomboList
from random import randrange
from tombola import Tombola

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
        
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))

In [29]:
print(issubclass(TomboList, Tombola))

t = TomboList(range(100))
print(isinstance(t, Tombola))

True
True


- 부모 클래스 확인
- `__mro__`: 메서드 결정 순서를 담은 특별 클래스 속성, 상속 계층을 보여줌

In [30]:
print(BingoCage.__bases__)
print(BingoCage.__mro__)

print(TomboList.__bases__)
print(TomboList.__mro__)

(<class 'tombola.Tombola'>,)
(<class '__main__.BingoCage'>, <class 'tombola.Tombola'>, <class 'abc.ABC'>, <class 'object'>)
(<class 'list'>,)
(<class '__main__.TomboList'>, <class 'list'>, <class 'object'>)


In [31]:
ConcreteTombola = TomboList


# 반복형에서 객체를 생성하고 로딩
balls = list(range(3))
globe = ConcreteTombola(balls)
print(globe.loaded())
print(globe.inspect())
    
# 공을 꺼내서 수집
picks = []
picks.append(globe.pick())
picks.append(globe.pick())
picks.append(globe.pick())

# 상태와 결과를 확인
print(globe.loaded())
print(sorted(picks) == balls)

# 다시 로딩
globe.load(balls)
print(globe.loaded())
picks = [globe.pick() for i in balls]
print(globe.loaded())

# 객체가 비어 있을 때 `LookupError` 또는 서브 예외가 발생하는지 확인
globe = ConcreteTombola([])
try:
    globe.pick()
except LookupError as exc:
    print('OK')
    
# 공 100개를 로딩하고 꺼내서 모두 제대로 나오는지 검증한다.
balls = list(range(100))
globe = ConcreteTombola(balls)
picks = []
while globe.inspect():
    picks.append(globe.pick())
print(len(picks) == len(balls))
print(set(picks) == set(balls))

# 순서가 변경되었는지, 단지 앞에서 뒤로 순서만 바꾼 것은 아닌지 확인한다.
print(picks != balls)
print(picks[::-1] != balls)

True
(0, 1, 2)
False
True
True
False
OK
True
True
True
True


## 11.8 Tombola 서브클래스 테스트 방법
- 그때그때 했습니다.

## 11.9 `register()`의 실제 용법
- 등록될 클래스 위에 데커레이터를 사용해서 정의할 수 있지만,
- 베이스 클래스이 정의된 파일에서, 다른 파일에 있는 클래스를 등록할 때 `register()`를 사용할 때가 있다.
- (예) `Tombola.register(TomboList)`

## 11.10 오리처럼 행동할 수 있는 거위
- `__subclasshook__()`이라는 특별 메서드를 정의해놓으면 ABC를 상속 또는 등록하지 않아도 서브 클래스로 인식할 수 있다. 
  - (예) `collections.abc.Sized`에는 `__subclasshook__()`이 정의되어 있다.
  - 주로, 특별 메서드 하나만 선언한 ABC에서만 `__subclasshook__()`를 정의한다. 해당 특별 메서드가 기대한 일이라고 확신할 수 있을 때 구현한다.
  - 매핑이 `__getitem__()`, `__len__()`, `__iter__()`을 구현해도 `Sequence`형이 되지 않는 이유는 매핑에서 구현한 메서드들은 시퀀스형과 다르게 작동하게 끔 정의하기 때문이다.

In [32]:
class Struggle:
    def __len__():
        pass
    
from collections import abc

print(isinstance(Struggle(), abc.Sized))
print(issubclass(Struggle, abc.Sized))

True
True
